In [1]:
from qdrant_client.http import models
from qdrant_client import QdrantClient
import time
import re
from semantic_text_splitter import TiktokenTextSplitter
from datasets import Dataset
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
import datetime
import logging
import uuid

In [2]:
#islam :
qdrant_client = QdrantClient(
    url="https://5c32ac64-b1f7-4665-91eb-e321a98c02f6.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Wd_RTregmznFMCyDLagJHM_7a5TjJJuFLVTuMgfjQD44-BHLnhYbUg",  # replace with your actual API key
)
collection_name = "News_source"

In [4]:
# rachid
qdrant_client = QdrantClient(
    url="https://3e4bc828-9c33-4845-bb01-1de4acb3675d.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="4ah-1sGbl4OUVUIVSeCtQvHaPIASSjqupe2KoG4G3zxYzkVNp-UJlA",
)
collection_name = "News_source"
guardian_api_key = "d69e575e-91d0-45c1-97f4-ac0fe69f9899"
duplicates_seuil=100
max_consecutive_same_articles=4
from_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
qdrant_client.recreate_collection(
    collection_name= collection_name,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE, on_disk=True),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,
            always_ram=False,
        ),
    ),
    hnsw_config=models.HnswConfig(on_disk=True),
    on_disk_payload=True,
)

C:\Users\rachi\AppData\Local\Temp\ipykernel_8468\193796489.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [7]:
def remove_tags(input_text):
        clean_text = re.sub('<.*?>', '', input_text)
        timing_pattern = r'\b\d{1,2}\.\d{2}[apmAPM]+\s+[GMTgmt]+\b' 
        clean_text = re.sub(timing_pattern, '', clean_text) 
        clean_text = re.sub(r'\s+', ' ', clean_text)
        clean_text = clean_text.replace('\n', ' ')
        pattern = r'© \d+ BBC\. The BBC is not responsible for the content of external sites\. Read about our approach to external linking\.'
        clean_text = re.sub(pattern, '', clean_text)
        return clean_text


def process_articles(collected_articles):
        chunked_articles = []
        chunked_titles = []
        chunked_pubdate=[]
        for article in collected_articles:  
            max_tokens = 512 
            splitter = TiktokenTextSplitter("gpt-3.5-turbo", trim_chunks=False)
            chunks = splitter.chunks(article['content'], max_tokens)
            for chunk in chunks:
                chunked_articles.append(chunk)
                chunked_titles.append(article['title'])
                chunked_pubdate.append(article['publishdate'])
        data_dict = {
            "title": chunked_titles,
            "content": chunked_articles,
            "publishdate": chunked_pubdate,
        }
        dataset = Dataset.from_dict(data_dict)
        return dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def _mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def embed_text(examples):
    inputs = tokenizer(
        examples["content"], padding=True, truncation=True, return_tensors="pt"
    )
    with torch.no_grad():
        model_output = model(**inputs)
    pooled_embeds = _mean_pooling(model_output, inputs["attention_mask"])
    return {"embedding": pooled_embeds.cpu().numpy()}

def generate_embeddings(dataset):
    return dataset.map(embed_text, batched=True, batch_size=128)


def _upload_documents_to_Qdrant(data, source):
        points = []
        for title, content, publishdate, embedding in zip(data["title"], data["content"], data["publishdate"], data["embedding"]):
            new_id = str(uuid.uuid4())  # Generate a new UUID for each document
            point = models.PointStruct(
                id=new_id,
                vector=embedding,
                payload={
                    "title": title,
                    "content": content,
                    "publishdate": publishdate,
                    "source" : source
                }
            )
            points.append(point)

        qdrant_client.upsert(
            collection_name='News_source',
            points=points
        )

        logging.info(f"Uploaded {len(data['embedding'])} documents to the Qdrant database")


def upload_to_Qdrant(data, batch_size=35, source=''):
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            _upload_documents_to_Qdrant(batch , source)
            logging.info(f"Uploaded {i + len(batch)} documents")

c:\Users\rachi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
def fetch_guardian_news():
        ''' 
        This function fetches news articles from the guardian api
        '''
        endpoint = 'https://content.guardianapis.com/search'
        params = {
            'api-key': guardian_api_key,
            'show-fields': 'headline,body',
            f'from-date': from_date,
            'to-date': datetime.datetime.now().strftime('%Y-%m-%d'),
            'order-by': 'newest',
            'page-size': 200
        }
        publish_dates=set() # to avoid duplicate articles
        count_duplicates_per_request=0
        collected_articles = []
        consecutive_same_articles = 0
        while True:
            if len(collected_articles) % 10 == 0: 
                print(f"Number of collected documents so far are : {len(collected_articles)} ....")
            response = requests.get(endpoint, params)
            data = response.json() 
            articles = data['response']['results']
            previous_article_count = len(collected_articles)
            for article in articles:
                title = article['fields']['headline']
                content = remove_tags(article['fields']['body'])
                publish_date=article['webPublicationDate']
                if publish_date not in publish_dates:
                        publish_dates.add(publish_date)
                        collected_articles.append({'title': title, 'content': content, 'publishdate':publish_date})
                else:
                    count_duplicates_per_request+=1
                    continue
            if previous_article_count == len(collected_articles):
                consecutive_same_articles += 1

            if consecutive_same_articles >= max_consecutive_same_articles:  # Replace with desired threshold
                break
            if count_duplicates_per_request>duplicates_seuil:
                print(f"Number of duplicate articles retrieved from the guardien news source is : {count_duplicates_per_request} , breaking ...")
                break
            time.sleep(1)
        return process_articles(collected_articles)

In [9]:
articles = fetch_guardian_news()
embeddings = generate_embeddings(articles)

Number of collected documents so far are : 0 ....
Number of duplicate articles retrieved from the guardien news source is : 224 , breaking ...


Map:   0%|          | 0/628 [00:00<?, ? examples/s]

In [10]:
upload_to_Qdrant(data=embeddings, source='guardian')

In [11]:
qdrant_client.get_collection(collection_name=collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=628, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=ScalarQu

In [12]:
def count_articles_by_source(client, collection_name, source_name, retries=5):
    count_result = client.count(
        collection_name=collection_name,
        count_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="source",
                    match=models.MatchValue(value=source_name),
                ),
            ]
        )
    )
    return count_result.count

In [13]:
count_articles_by_source(qdrant_client, collection_name, "guardian")

628